In [ ]:
# ✅ Step 0: Install necessary libraries
!pip install -q transformers datasets accelerate

# ✅ Step 1: Add swap memory to avoid RAM crash (optional but recommended)
!fallocate -l 4G /swapfile
!chmod 600 /swapfile
!mkswap /swapfile
!swapon /swapfile

# ✅ Step 2: Import libraries
import pandas as pd
from datasets import Dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM

# ✅ Step 3: Load and preprocess CSV data
df = pd.read_csv("/content/sample_data/Copy of train.csv")
df = df[["text"]].dropna()
df["text"] = df["text"].astype(str).str.strip()
dataset = Dataset.from_pandas(df[["text"]])

# ✅ Step 4: Convert to Zephyr chat format
def to_chat_format(example):
    return {
        "chat": [
            {"role": "user", "content": example["text"]},
            {"role": "assistant", "content": ""}
        ]
    }

dataset = dataset.map(to_chat_format)

# ✅ Step 5: Load model and tokenizer
model_name = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")
import torch
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16  # or "auto"
).to("cuda")

# ✅ Step 6: Tokenize using apply_chat_template
def tokenize_chat(example):
    # 1. Format the chat as a string
    chat_str = tokenizer.apply_chat_template(
        example["chat"],
        tokenize=False,  # We just want a formatted string first
        add_generation_prompt=False
    )
    # 2. Tokenize the resulting string
    tokens = tokenizer(
        chat_str,
        truncation=True,
        max_length=1024,
        padding=False
    )
    return tokens



# ✅ Step 7: Tokenize + Save to Disk (prevents RAM crash)
tokenized_dataset = dataset.map(tokenize_chat, remove_columns=dataset.column_names)
tokenized_dataset.save_to_disk("tokenized_dataset")

# ✅ Step 8: (RECOMMENDED) Restart runtime here to clear memory
# Runtime > Restart Runtime

# ✅ Step 9: Reload saved tokenized dataset
from datasets import load_from_disk
tokenized_dataset = load_from_disk("tokenized_dataset")

# ✅ Step 10: Set up training
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

model.gradient_checkpointing_enable()

training_args = TrainingArguments(
    output_dir="./zephyr-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    learning_rate=2e-5,
    logging_steps=10,
    save_steps=200,
    save_total_limit=2,
    fp16=True,  # Use bf16=True if you have an A100 or similar
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# ✅ Step 11: Train
trainer.train()


mkswap: /swapfile: warning: wiping old swap signature.
Setting up swapspace version 1, size = 4 GiB (4294963200 bytes)
no label, UUID=088174d9-132b-4e26-baaf-9d328309377b
swapon: /swapfile: swapon failed: Invalid argument


<ipython-input-1-e95502c4174c>:16: DtypeWarning: Columns (225,226,227,228,229,230,248,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,47

Map:   0%|          | 0/1381 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]